In [43]:
#This program loads the update client message type into its related transaction detail table inside of the blockchain db
import pandas as pd
import json
import psycopg2
import logging

DB_NAME = 'blockchain'
DB_USER = 'postgres'
DB_PASSWORD = 'auth2020'
DB_HOST = 'localhost'
DB_PORT = '5432'


try:
    # Create a PostgreSQL connection
    conn = psycopg2.connect(
        dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST, port=DB_PORT
    )
    cur = conn.cursor()
    print("Connection established.")  # Log successful connection (you can modify this)

    # Configure logging to write to a log file
    logging.basicConfig(filename='log.txt', level=logging.INFO)
    logging.info("Connection established with Database.")  # Log the same message to the file


    # Additional code for database operations goes here...

except Exception as err:
    # Handle exceptions
    print("Unable to connect to database.")  # Log error (you can modify this)
    print(err)  # Print the specific error message

    # Optionally, write the error message to a file (error.txt)
    with open('error.txt', 'a') as error_file:
        error_file.write(str(err) + '\n')
        

Connection established.


In [44]:
import os
import logging

block = '6041212'  # Change this number if you want to load a different file
file_path = f'C:/Users/lmhmo/Indexer_Project/indexer_project/Decrypted_files/{block}.json'

try:
    if os.path.exists(file_path):
        print(f"File '{file_path}' exists. Proceeding with further actions.")
        # Add your additional code here

        # Configure logging to write to both log.txt and error.txt
        logging.basicConfig(filename='log.txt', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
        logging.info(f"File '{file_path}' exists. Proceeding with further actions.")

    else:
        print(f"File '{file_path}' does not exist.")
        # Log the error message to error.txt
        with open('error.txt', 'a') as error_file:
            error_file.write(f"File '{file_path}' does not exist.\n")

except Exception as e:
    print(f"An error occurred: {e}")
    # Log the error message to error.txt
    with open('error.txt', 'a') as error_file:
        error_file.write(f"An error occurred: {e}\n")
        
       

File 'C:/Users/lmhmo/Indexer_Project/indexer_project/Decrypted_files/6041212.json' exists. Proceeding with further actions.


In [45]:
import psycopg2

def get_most_recent_transaction_id(conn):
    try:
        with conn.cursor() as cursor:
            # Query to retrieve the most recent transaction_begin_redelegate_id
            get_transaction_id_query = """
                SELECT transaction_delegate_id
                FROM transaction_header_delegate
                ORDER BY transaction_delegate_id DESC
                LIMIT 1;
            """
            cursor.execute(get_transaction_id_query)
            result = cursor.fetchone()
            if result:
                transaction_id = result[0]
                print(f"Most recent transaction_execute_id: {transaction_id}")
                return transaction_id
            else:
                print("No transaction_execute_id found.")
                return None
    except psycopg2.Error as e:
        print(f"Error retrieving transaction_execute_id: {e}")
        return None

# Example usage: Replace with your actual database connection details
db_params = {
    "host": "localhost",
    "database": "blockchain",
    "user": "postgres",
    "password": "auth2020"
}

try:
    conn = psycopg2.connect(**db_params)
    get_most_recent_transaction_id(conn)
    conn.close()
except psycopg2.Error as e:
    print(f"Error connecting to the database: {e}")
       

Most recent transaction_execute_id: cc42a26d-a6dd-4a27-bc7e-230be3a059d6


In [46]:
try:
    conn = psycopg2.connect(**db_params)
    recent_transaction_id = get_most_recent_transaction_id(conn)
    conn.close()

    if recent_transaction_id:
        # Continue with your existing code
        with open(file_path, 'r') as json_file:
            json_data = json.load(json_file)

            # Create DataFrames for delegator_address, validator_address, denom, and amount
            delegator_address = pd.DataFrame({"delegator_address": [json_data["tx"]["body"]["messages"][0]["delegator_address"]]})
            validator_address = pd.DataFrame({"validator_address": [json_data["tx"]["body"]["messages"][0]["validator_address"]]})
            denom = json_data["tx"]["body"]["messages"][0]["amount"]["denom"]
            amount = json_data["tx"]["body"]["messages"][0]["amount"]["amount"]


            table_name = 'transaction_detail_msg_delegate'

            # Construct the combined insert query
            msg_insert_query = f"""
                 INSERT INTO {table_name} (delegator_address, validator_address, denom, amount, transaction_delegate_id)
                VALUES ('{delegator_address}', '{validator_address}', '{denom}', '{amount}', '{recent_transaction_id}');
            """

            # Replace with your actual database connection parameters
            conn = psycopg2.connect(
                dbname='blockchain',
                user='postgres',
                password='auth2020',
                host='localhost',
                port='5432'
            )

            with conn:
                with conn.cursor() as cur:
                    cur.execute(msg_insert_query)

            print("Data inserted successfully.")
    else:
        print("No recent transaction ID found.")
except psycopg2.Error as e:
    print(f"Error connecting to the database: {e}")
    
    

Most recent transaction_execute_id: cc42a26d-a6dd-4a27-bc7e-230be3a059d6
Data inserted successfully.
